In [0]:
/*
CREATE TABLE gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE (
    CPPROP_SOURCE_ID VARCHAR(400),
    CPPROP_PARENT_SOURCE_ID VARCHAR(400),
    CASE_SOURCE_ID VARCHAR(120),
    CPPTP_SOURCE_ID VARCHAR(400),
    ATTR_FP VARCHAR(400),
    CASETYPE VARCHAR(120),
    ATTR_VALUE VARCHAR(32000),
    ATTR_BVALUE STRING,
    ATTR_TYPE VARCHAR(120),
    STRUCTURETYPE VARCHAR(400),
    MAPPING VARCHAR(400),
    TIMESTAMP TIMESTAMP
);
*/

/*
CREATE TABLE gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES (
    CPPROP_SOURCE_ID VARCHAR(400),
    CPPROP_PARENT_SOURCE_ID VARCHAR(400),
    CASE_SOURCE_ID VARCHAR(120),
    CPPTP_SOURCE_ID VARCHAR(400),
    ATTR_FP VARCHAR(400),
    CASETYPE VARCHAR(120),
    ATTR_VALUE VARCHAR(32000),
    ATTR_BVALUE STRING,
    ATTR_TYPE VARCHAR(120),
    STRUCTURETYPE VARCHAR(400),
    MAPPING VARCHAR(400),
    CTP_KEY BIGINT
);
*/
--execute immediate('Truncate table ADS_ETL_OWNER.STG_SMA_CASE_PHASE_PROPERTIES');
--execute immediate('Truncate table ADS_ETL_OWNER.STG_SMA_CASE_PHASE_PROPERTIES_PARSE');

/*select SYS_EFFECTIVE_DATE, count(1)
from gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full j 
group by SYS_EFFECTIVE_DATE;*/
-- 2025-09-01,2025-09-02,2025-09-03


/* STG_SMA_CASE_PHASE_PROPERTIES_parse */
/* 1.run 7m37s, 642.242.223 rows */
insert into gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE 
SELECT 
    UPPER(attr_fp) || '.' || case_cidla AS cpprop_source_id,
    CASE 
        WHEN LENGTH(attr_fp) - LENGTH(REPLACE(attr_fp, '.', '')) > 0 
        THEN SUBSTRING(attr_fp, 1, LENGTH(attr_fp) - INSTR(REVERSE(attr_fp), '.')) || '.' || case_cidla
        ELSE NULL 
    END AS cpprop_parent_source_id,
    case_cidla AS case_source_id,
    UPPER(attr_fp) || '|' || casetype AS cpptp_source_id,
    UPPER(attr_fp) AS attr_fp,
    casetype,
    attr_value,
    attr_bvalue,
    attr_type,
    structuretype,
    mapping,
    timestamp
FROM (
    SELECT 
        att1.eventid,
        att1.case_cidla,
        att1.casetype,
        att1.timestamp,
        att1.attr_id,
        UPPER(att1.attr_fp) AS Attr_FP,
        UPPER(att1.attr_name) AS Attr_name,
        att1.attr_value,
        att1.attr_bvalue,
        att1.attr_type,
        att1.parentid,
        att1.modifiedby,
        att1.structuretype,
        att1.mapping
    FROM (
        SELECT 
            att0.eventid,
            att0.type,
            att0.case_cidla,
            att0.casetype,
            att0.timestamp,
            attr.id AS attr_id,
            attr.fullPath AS attr_fp,
            attr.name AS attr_name,
            attr.value AS attr_value,
            attr.blobValue AS attr_bvalue,
            attr.attributeType AS attr_type,
            attr.parentId AS parentid,
            attr.modifiedBy AS modifiedby,
            attr.structureType AS structuretype,
            attr.mapping AS mapping
        FROM (
            SELECT 
                j.eventid,
                j.cidla AS case_cidla,
                CASE 
                    WHEN j.casetype LIKE '% %' THEN 'ERR' 
                    ELSE j.casetype 
                END AS casetype,
                j.type,
                j.timestamp,
                jsondata
            FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full j
            WHERE SYS_EFFECTIVE_DATE = '2025-09-01'
                AND j.type IN (
                    'CASE_CREATED',
                    'CASE_COMPLETED',
                    'CASE_WAITING',
                    'CASE_RUNNING',
                    'CASE_UPDATED',
                    'CASE_PHASE_CHANGED',
                    'CASE_RECONCILE'
                )
                AND FROM_UNIXTIME(
                    CAST(get_json_object(jsondata, '$.theCase.createdAt') AS BIGINT) / 1000
                ) >= ADD_MONTHS(DATE_TRUNC('day', CURRENT_DATE()), -25)
        ) att0
        LATERAL VIEW EXPLODE(
            FROM_JSON(
                get_json_object(jsondata, '$.theCase.attributes'),
                'array<struct<id:string,fullPath:string,name:string,value:string,blobValue:string,attributeType:string,parentId:string,modifiedBy:string,structureType:string,mapping:string>>'
            )
        ) attr_table AS attr
    ) att1
) 
/*  select 
    upper(attr_fp)||'.'||case_cidla cpprop_source_id,
  case when INSTR(attr_fp, '.', -1)>0 then SUBSTR (attr_fp,
                       0,
                       (INSTR (attr_fp, '.', -1)) - 1) ||'.'||case_cidla
                       else null end  cpprop_parent_source_id,
    CASE_CIDLA as case_source_id,
    upper(attr_fp)||'|'||casetype as cpptp_source_id ,
  upper(ATTR_FP) as attr_fp ,
  CASETYPE   ,
  ATTR_VALUE  ,
  ATTR_BVALUE  ,
  ATTR_TYPE  ,
  STRUCTURETYPE ,
  MAPPING, 
  timestamp
   from (
  select 
    att1.eventid,
  att1.case_cidla,
  att1.casetype,
  att1.timestamp,
  att1.attr_id,
  upper(att1.attr_fp) Attr_FP,
  upper(att1.attr_name) Attr_name,
  att1.attr_value,
  att1.attr_bvalue,
  att1.attr_type,
  att1.parentid,
  --row_number() over (partition by  upper(att1.attr_fp),att1.case_cidla order by att1.timestamp desc) as rn,
  att1.modifiedby, 
  att1.structuretype,  
  att1.mapping
  from (select 
       att0.eventid,
       att0.type,
       att0.case_cidla,
       att0.casetype,
       att0.timestamp,
      attr.*--,
     -- row_number() over (partition by att0.case_cidla, attr.attr_fp order by timestamp desc)
      from (select 
       j.eventid,
       j.cidla as case_cidla,
       case when j.casetype like '% %' then 'ERR' else j.casetype end as casetype,
       j.type,
       j.timestamp,
       jsondata
      from  gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full j
      where
      SYS_EFFECTIVE_DATE = '2025-09-01'
      --CASE filter
      and  j.type in ('CASE_CREATED',
    'CASE_COMPLETED',
    'CASE_WAITING',
    'CASE_RUNNING',
    'CASE_UPDATED',
    'CASE_PHASE_CHANGED',
    'CASE_RECONCILE'
--,     --JVE:  zmena typu case
          )
       and unix_ts_to_date(json_value(jsondata, '$.theCase.createdAt')/1000) >= ADD_MONTHS(trunc(SYSDATE), -25) 
          )att0 
      ,
      json_table(jsondata, '$.theCase.attributes[*]' 
      columns (attr_id path '$.id',
      attr_fp path '$.fullPath',
      attr_name path '$.name',
      attr_value path '$.value',
      attr_bvalue clob path '$.blobValue',
      attr_type path '$.attributeType',
      parentid path '$.parentId',
      modifiedby path '$.modifiedBy',
      structuretype path '$.structureType',
      mapping path '$.mapping')
      ) attr) att1)
  */
 ;

/* stg_sma_case_phase_properties */
/* 1.run 3m40s */
insert  into  gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES
select CPPROP_SOURCE_ID   
 , CPPROP_PARENT_SOURCE_ID 
 , CASE_SOURCE_ID     
 , CPPTP_SOURCE_ID   
 , ATTR_FP         
 , CASETYPE         
 , ATTR_VALUE    
 --,cpprop_value_text  
 , ATTR_BVALUE         
 , ATTR_TYPE           
 , STRUCTURETYPE           
 , MAPPING      
 , ct.ctp_key           
 from (select c.*, row_number() over (partition by cpprop_source_id order by timestamp desc) as rn
 from gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE c) src
 join gap_catalog.ads_src_20250901.case_types ct on ct.ctp_source_id=src.casetype and ct.ctp_source_sys_origin='RDS_CASETYPE' and ct.ctp_valid_to=date'3000-01-01' 
 where rn=1;

 select * from gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE 



